<a href="https://colab.research.google.com/github/KashanHumayun/how-brainlike-is-deepseek/blob/main/01_RSA_PaperReady_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paper-Ready RSA Pipeline: GPT‑2 vs DeepSeek‑7B vs LLaMA‑2‑7B

This notebook provides a clean, reproducible RSA pipeline with subject-level stats, permutation tests, and noise ceiling.

## 1) Configuration & Paths

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path
import numpy as np
import torch

# Use the same base directory used during download and extraction
BASE_DIR = Path('/content/drive/MyDrive/pereira2018')

# All data references now point into Drive
DATA_DIR = BASE_DIR
RAW_DIR = BASE_DIR / 'pereira2018_raw'
RESULTS_DIR = BASE_DIR / 'results'; RESULTS_DIR.mkdir(exist_ok=True)

# Brain RSM derived files saved in Drive as well
BRAIN_RSM_PATH = BASE_DIR / 'brain_rsm_expt2_group.npy'
PER_SUBJECT_RSMS_PATH = BASE_DIR / 'brain_rsms_expt2_per_subject.npy'
SENTENCES_PATH = BASE_DIR / 'stimuli_384sentences.txt'

print('Base dir   :', BASE_DIR.resolve())
print('Raw dir    :', RAW_DIR.resolve(), RAW_DIR.exists())
print('Results dir:', RESULTS_DIR.resolve())
print('Brain RSM  :', BRAIN_RSM_PATH.exists(), '->', BRAIN_RSM_PATH)
print('Per-subj   :', PER_SUBJECT_RSMS_PATH.exists(), '->', PER_SUBJECT_RSMS_PATH)
print('Sentences  :', SENTENCES_PATH.exists(), '->', SENTENCES_PATH)

def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')

DEVICE = get_device()
DEVICE


Base dir   : /content/drive/MyDrive/pereira2018
Raw dir    : /content/drive/MyDrive/pereira2018/pereira2018_raw True
Results dir: /content/drive/MyDrive/pereira2018/results
Brain RSM  : True -> /content/drive/MyDrive/pereira2018/brain_rsm_expt2_group.npy
Per-subj   : True -> /content/drive/MyDrive/pereira2018/brain_rsms_expt2_per_subject.npy
Sentences  : True -> /content/drive/MyDrive/pereira2018/stimuli_384sentences.txt


device(type='cpu')

## 2) Utilities: RSM, RSA, loaders, stats

In [3]:

import scipy.io
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np

def compute_cosine_rsm(X):
    D = pdist(X, metric='cosine')
    S = 1 - squareform(D)
    np.fill_diagonal(S, 1.0)
    return S

def _is_numeric_matrix(arr):
    return isinstance(arr, np.ndarray) and arr.ndim == 2 and np.issubdtype(arr.dtype, np.number)

def _iter_arrays(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k.startswith('__'): continue
            yield from _iter_arrays(v)
    elif isinstance(obj, np.ndarray):
        if _is_numeric_matrix(obj):
            yield obj
        else:
            if obj.dtype == object or obj.ndim > 2:
                for x in obj.flat:
                    try: yield from _iter_arrays(x)
                    except: pass
    elif isinstance(obj, (list, tuple)):
        for x in obj: yield from _iter_arrays(x)
    if isinstance(obj, np.ndarray) and obj.dtype.names is not None:
        for name in obj.dtype.names:
            try: yield from _iter_arrays(obj[name])
            except: pass

def extract_384_matrix(mat_path):
    data = scipy.io.loadmat(mat_path, squeeze_me=True, struct_as_record=False)
    cand = []
    for arr in _iter_arrays(data):
        if arr.ndim==2 and 384 in arr.shape:
            X = arr if arr.shape[0]==384 else arr.T
            if X.shape[0]==384 and X.shape[1] > 100:
                cand.append(X.astype(np.float32, copy=False))
    if not cand:
        raise ValueError(f'No 384×V array in {mat_path}')
    return max(cand, key=lambda a: a.shape[1])

def ensure_brain_rsms(RAW_DIR, BRAIN_RSM_PATH, PER_SUBJECT_RSMS_PATH):
    if BRAIN_RSM_PATH.exists() and PER_SUBJECT_RSMS_PATH.exists():
        print('Brain RSMs exist.'); return
    if not RAW_DIR.exists():
        raise FileNotFoundError('RAW_DIR missing; run download.')
    rsms = []
    for sdir in [d for d in RAW_DIR.glob('*') if d.is_dir()]:
        mat = next((p for p in [sdir/'examples_384sentences.mat', sdir/'data_384sentences.mat'] if p.exists()), None)
        if not mat:
            print('[WARN] missing .mat for', sdir.name); continue
        X = extract_384_matrix(mat)
        rsms.append(compute_cosine_rsm(X))
        print('RSM ok:', sdir.name, X.shape)
    rsms = np.stack(rsms)
    np.save(PER_SUBJECT_RSMS_PATH, rsms)
    np.save(BRAIN_RSM_PATH, rsms.mean(axis=0))
    print('Saved per-subject and group RSMs.')

def permutation_pvals(model_rsm, subj_rsms, n_perm=2000, seed=0):
    tri = np.triu_indices(model_rsm.shape[0], 1)
    obs = np.array([spearmanr(sr[tri], model_rsm[tri]).statistic for sr in subj_rsms])
    rng = np.random.default_rng(seed)
    n = model_rsm.shape[0]
    null = []
    for _ in range(n_perm):
        p = rng.permutation(n)
        perm = model_rsm[p][:, p]
        null.append([spearmanr(sr[tri], perm[tri]).statistic for sr in subj_rsms])
    null = np.array(null)
    pvals = ((np.abs(null) >= np.abs(obs)).mean(axis=0)).clip(1/n_perm, 1)
    return obs, pvals

def fdr_bh(p, alpha=0.05):
    p = np.asarray(p); idx = np.argsort(p); m = len(p)
    thresh = alpha * (np.arange(1, m+1)/m)
    passed = p[idx] <= thresh
    cut = np.where(passed)[0].max() if passed.any() else -1
    sig = np.zeros_like(p, bool)
    if cut >= 0: sig[idx[:cut+1]] = True
    return sig


In [4]:
# Robust validator + repair + RSM build wrapper for Pereira2018 in Colab with Drive persistence.

import os, time, tarfile, shutil, requests
from pathlib import Path
from typing import Optional, Tuple, List
import numpy as np
import scipy.io as sio

# ===== Paths consistent with your notebook =====
BASE_DIR = Path('/content/drive/MyDrive/pereira2018')
RAW_DIR  = BASE_DIR / 'pereira2018_raw'
BRAIN_RSM_PATH = BASE_DIR / 'brain_rsm_expt2_group.npy'
PER_SUBJECT_RSMS_PATH = BASE_DIR / 'brain_rsms_expt2_per_subject.npy'
RAW_DIR.mkdir(parents=True, exist_ok=True)

# ===== Subjects and source URLs =====
SUBJECTS = {
    'P01': 'https://www.dropbox.com/s/5duv9fgigrzx817/P01.tar?dl=1',
}
# ===== Helpers: download, safe extract, validation =====
def download(url: str, out: Path, chunk=2**20, retries=4, timeout=120):
    out.parent.mkdir(parents=True, exist_ok=True)
    if out.exists() and out.stat().st_size > 0:
        return out
    last_err = None
    for attempt in range(1, retries+1):
        try:
            with requests.get(url, stream=True, timeout=timeout) as r:
                r.raise_for_status()
                with open(out, 'wb') as f:
                    for b in r.iter_content(chunk_size=chunk):
                        if b:
                            f.write(b)
            if out.stat().st_size == 0:
                raise RuntimeError('Empty file after download')
            return out
        except Exception as e:
            last_err = e
            time.sleep(2*attempt)
    raise RuntimeError(f'Failed to download {url}: {last_err}')

def safe_extract(tar_path: Path, dest_dir: Path):
    mode = 'r:*'  # supports plain .tar and compressed variants
    with tarfile.open(tar_path, mode) as tar:
        base = dest_dir.resolve()
        for m in tar.getmembers():
            p = (dest_dir / m.name).resolve()
            if not str(p).startswith(str(base)):
                raise Exception(f'Unsafe path in tar: {m.name}')
        tar.extractall(path=dest_dir)

def expected_mat_paths(subj_dir: Path) -> Tuple[Path, Path]:
    return subj_dir / 'examples_384sentences.mat', subj_dir / 'data_384sentences.mat'

def read_mat_header(p: Path) -> Optional[str]:
    if not p.exists():
        return None
    try:
        with open(p, 'rb') as f:
            head = f.read(128)
        if b'MATLAB 5.0 MAT-file' in head:
            return 'v5'
        if head.startswith(b'\x89HDF\r\n\x1a\n'):
            return 'v7.3'
        return None
    except Exception:
        return None

def quick_loadmat(p: Path) -> bool:
    try:
        # This catches truncation without spending time parsing all variables
        sio.loadmat(p, verify_compressed_data_integrity=False, variable_names=[])
        return True
    except Exception:
        return False

def validate_subject(subj: str) -> bool:
    subj_dir = RAW_DIR / subj
    m1, m2 = expected_mat_paths(subj_dir)
    mats = [p for p in (m1, m2) if p.exists()]
    if not mats:
        print(f'[{subj}] No expected .mat files found.')
        return False
    ok_any = False
    for p in mats:
        hdr = read_mat_header(p)
        ok = hdr in ('v5', 'v7.3') and quick_loadmat(p)
        print(f'[{subj}] {p.name}: header={hdr} loadable={ok}')
        ok_any |= ok
    return ok_any

def repair_subject(subj: str, url: str):
    subj_dir = RAW_DIR / subj
    tar_path = RAW_DIR / f'{subj}.tar'
    if subj_dir.exists():
        shutil.rmtree(subj_dir)
    if tar_path.exists():
        tar_path.unlink()
    print(f'Re-downloading {subj}...')
    download(url, tar_path)
    print(f'Extracting {subj}...')
    safe_extract(tar_path, RAW_DIR)
    # Optionally free Drive space by removing the tar after extraction
    # tar_path.unlink(missing_ok=True)

def detect_failed_subjects(raw_dir: Path) -> List[str]:
    failed = []
    for subj_dir in sorted(raw_dir.glob('[PM][0-9][0-9]')):
        candidates = [subj_dir/'examples_384sentences.mat', subj_dir/'data_384sentences.mat']
        bad = False
        have_any = False
        for p in candidates:
            if p.exists():
                have_any = True
                if not (read_mat_header(p) in ('v5', 'v7.3') and quick_loadmat(p)):
                    bad = True
                    break
        if bad or not have_any:
            failed.append(subj_dir.name)
    return failed

# ===== Wrapper around your ensure_brain_rsms =====
def ensure_rsms_with_repair(raw_dir: Path, out_group: Path, out_per_subj: Path):
    try:
        ensure_brain_rsms(raw_dir, out_group, out_per_subj)
        return
    except OSError as e:
        print('[WARN] RSM build failed while reading .mat. Attempting targeted repair...')
        failed_subjects = detect_failed_subjects(raw_dir)
        if not failed_subjects:
            # If nothing identified, assume all known subjects might be affected
            failed_subjects = list(SUBJECTS.keys())
        print('Subjects marked for repair:', failed_subjects)
        for sid in failed_subjects:
            if sid in SUBJECTS:
                repair_subject(sid, SUBJECTS[sid])
            else:
                print(f'[INFO] No download URL registered for {sid}. Skipping.')
        # Validate again before retry
        for sid in failed_subjects:
            ok = validate_subject(sid)
            if not ok:
                raise RuntimeError(f'{sid} still failing validation after repair.')
        # Retry once
        ensure_brain_rsms(raw_dir, out_group, out_per_subj)

# ===== Execute: validate, repair if needed, then build and load =====
# Optional pre-validation for visibility
for sid in SUBJECTS:
    _ = validate_subject(sid)

ensure_rsms_with_repair(RAW_DIR, BRAIN_RSM_PATH, PER_SUBJECT_RSMS_PATH)

group_rsm = np.load(BRAIN_RSM_PATH)
per_subj_rsms = np.load(PER_SUBJECT_RSMS_PATH)
print('group_rsm:', group_rsm.shape, '| per_subj_rsms:', per_subj_rsms.shape)


[P01] No expected .mat files found.
Brain RSMs exist.
group_rsm: (384, 384) | per_subj_rsms: (7, 384, 384)


In [5]:
# === Pereira2018: robust brain RSM builder (Colab + Drive, no re-downloads required) ===
# It scans /content/drive/MyDrive/pereira2018/pereira2018_raw for subject folders,
# robustly reads the MATLAB files, constructs per-subject cosine RSMs, saves both the
# per-subject stack and the group mean to Drive, and prints their shapes.

from pathlib import Path
import numpy as np
import scipy.io as sio

# --- Optional: mount Drive if running in Colab ---
def _in_colab():
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False

if _in_colab():
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=False)

# --- Paths ---
BASE_DIR = Path('/content/drive/MyDrive/pereira2018')
RAW_DIR  = BASE_DIR / 'pereira2018_raw'
BRAIN_RSM_PATH = BASE_DIR / 'brain_rsm_expt2_group.npy'
PER_SUBJECT_RSMS_PATH = BASE_DIR / 'brain_rsms_expt2_per_subject.npy'

print('BASE_DIR :', BASE_DIR.resolve())
print('RAW_DIR  :', RAW_DIR.resolve(), '| exists:', RAW_DIR.exists())

# --- Robust .mat loading utilities ---
def robust_loadmat(mat_path: Path):
    """
    Load MATLAB file while skipping zlib CRC integrity checks to avoid
    'OSError: could not read bytes' on compressed variables.
    """
    return sio.loadmat(
        mat_path,
        simplify_cells=True,
        squeeze_me=False,
        struct_as_record=False,
        verify_compressed_data_integrity=False,  # critical for some files
    )

def _collect_arrays(obj, out_list):
    """Depth-first traversal collecting 2D numpy arrays that contain a 384 dimension."""
    if isinstance(obj, np.ndarray):
        if obj.ndim == 2 and 384 in obj.shape:
            out_list.append(obj)
    elif isinstance(obj, dict):
        for v in obj.values():
            _collect_arrays(v, out_list)
    elif isinstance(obj, (list, tuple)):
        for v in obj:
            _collect_arrays(v, out_list)

def find_384_matrix(mdict) -> np.ndarray:
    """
    Search the loaded MATLAB dict for a matrix with one dimension equal to 384.
    Returns an array of shape (384, V) as float32.
    Preference is given to arrays already shaped (384, V).
    """
    candidates = []
    for k, v in mdict.items():
        if not str(k).startswith('__'):
            _collect_arrays(v, candidates)
    if not candidates:
        raise ValueError("No 2D array containing dimension 384 found in this .mat file.")

    # Score: prefer those with 384 as first dimension, then largest width.
    def score(a):
        return (a.shape[0] == 384, max(a.shape))
    candidates.sort(key=score, reverse=True)

    X = candidates[0]
    X = X.astype(np.float32, copy=False)
    return X if X.shape[0] == 384 else X.T

def compute_cosine_rsm(X: np.ndarray) -> np.ndarray:
    """
    X: shape (N=384, D). Returns cosine-similarity RSM of shape (384, 384).
    """
    Xn = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-8)
    R = Xn @ Xn.T
    return np.clip(R, -1.0, 1.0).astype(np.float32)

def build_brain_rsms_from_raw(raw_dir: Path,
                              out_group: Path,
                              out_per_subj: Path):
    """
    Iterate subject folders in raw_dir, read their MATLAB files,
    compute per-subject RSMs, and save both per-subject and group RSMs.
    """
    if not raw_dir.exists():
        raise RuntimeError(f"RAW_DIR not found: {raw_dir}")

    # Limit to canonical subject folder names like P01, M02, etc.
    subj_dirs = sorted([d for d in raw_dir.iterdir() if d.is_dir() and d.name[:1] in ('P','M') and len(d.name) >= 3])
    if not subj_dirs:
        raise RuntimeError(f"No subject directories found in {raw_dir}")

    per_subj = []
    included = []
    skipped  = []

    for subj_dir in subj_dirs:
        # Prefer examples_384sentences.mat if present, else data_384sentences.mat
        m1 = subj_dir / 'examples_384sentences.mat'
        m2 = subj_dir / 'data_384sentences.mat'
        mat_path = m1 if m1.exists() else m2 if m2.exists() else None

        if mat_path is None:
            print(f"[WARN] {subj_dir.name}: missing .mat file, skipping.")
            skipped.append(subj_dir.name)
            continue

        try:
            mdict = robust_loadmat(mat_path)
            X = find_384_matrix(mdict)   # (384, V)
            rsm = compute_cosine_rsm(X)  # (384, 384)
            per_subj.append(rsm)
            included.append(subj_dir.name)
            print(f"[OK] {subj_dir.name}: RSM built from {mat_path.name}  -> X: {X.shape}")
        except Exception as e:
            print(f"[FAIL] {subj_dir.name}: {e}")
            skipped.append(subj_dir.name)

    if not per_subj:
        raise RuntimeError("Failed to build any per-subject RSMs.")

    per_subj = np.stack(per_subj, axis=0)         # (S, 384, 384)
    group = per_subj.mean(axis=0, dtype=np.float32)

    out_per_subj.parent.mkdir(parents=True, exist_ok=True)
    np.save(out_per_subj, per_subj)
    np.save(out_group, group)

    print("\n=== Summary ===")
    print(f"Subjects included ({len(included)}): {included}")
    if skipped:
        print(f"Subjects skipped   ({len(skipped)}): {skipped}")
    print(f"Saved per-subject RSMs -> {out_per_subj}  shape={per_subj.shape}")
    print(f"Saved group RSM       -> {out_group}      shape={group.shape}")

# --- Execute end-to-end ---
build_brain_rsms_from_raw(RAW_DIR, BRAIN_RSM_PATH, PER_SUBJECT_RSMS_PATH)

# --- Load for confirmation ---
group_rsm = np.load(BRAIN_RSM_PATH)
per_subj_rsms = np.load(PER_SUBJECT_RSMS_PATH)
print('\nLoaded:')
print('group_rsm:', group_rsm.shape, '| per_subj_rsms:', per_subj_rsms.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BASE_DIR : /content/drive/MyDrive/pereira2018
RAW_DIR  : /content/drive/MyDrive/pereira2018/pereira2018_raw | exists: True
[OK] M02: RSM built from examples_384sentences.mat  -> X: (384, 170712)
[OK] M04: RSM built from examples_384sentences.mat  -> X: (384, 195127)
[OK] M07: RSM built from examples_384sentences.mat  -> X: (384, 185866)
[OK] M08: RSM built from examples_384sentences.mat  -> X: (384, 177341)
[OK] M09: RSM built from examples_384sentences.mat  -> X: (384, 145303)
[OK] M14: RSM built from examples_384sentences.mat  -> X: (384, 182556)
[OK] M15: RSM built from examples_384sentences.mat  -> X: (384, 185703)

=== Summary ===
Subjects included (7): ['M02', 'M04', 'M07', 'M08', 'M09', 'M14', 'M15']
Saved per-subject RSMs -> /content/drive/MyDrive/pereira2018/brain_rsms_expt2_per_subject.npy  shape=(7, 384, 384)
Saved group RSM       -> /content/drive

## 3) Build/Load Brain RSMs

In [6]:
from pathlib import Path
import numpy as np

BASE_DIR = Path('/content/drive/MyDrive/pereira2018')
RAW_DIR  = BASE_DIR / 'pereira2018_raw'
BRAIN_RSM_PATH = BASE_DIR / 'brain_rsm_expt2_group.npy'
PER_SUBJECT_RSMS_PATH = BASE_DIR / 'brain_rsms_expt2_per_subject.npy'

def ensure_rsms_with_repair(raw_dir: Path, out_group: Path, out_per_subj: Path):
    try:
        ensure_brain_rsms(raw_dir, out_group, out_per_subj)
        return
    except OSError as e:
        # Typical message: "could not read bytes" from scipy zlib stream
        print('[WARN] RSM build failed while reading .mat. Attempting targeted repair...')
        # Identify which subject fails by probing loadmat on each expected file
        import scipy.io as sio
        import glob
        failed_subjects = []
        for subj_dir in sorted(raw_dir.glob('[PM][0-9][0-9]')):
            candidates = [subj_dir/'examples_384sentences.mat', subj_dir/'data_384sentences.mat']
            for p in candidates:
                if p.exists():
                    try:
                        sio.loadmat(p, verify_compressed_data_integrity=False, variable_names=[])
                    except Exception:
                        failed_subjects.append(subj_dir.name)
                        break
        # If none specifically detected, fall back to validating all
        if not failed_subjects:
            failed_subjects = [d.name for d in sorted(raw_dir.glob('[PM][0-9][0-9]'))]
        # Repair those subjects using the URLs
        for sid in failed_subjects:
            if sid in SUBJECTS:
                repair_subject(sid, SUBJECTS[sid])
        # Retry once
        ensure_brain_rsms(raw_dir, out_group, out_per_subj)

# Run the defensive builder
ensure_rsms_with_repair(RAW_DIR, BRAIN_RSM_PATH, PER_SUBJECT_RSMS_PATH)

# Load and report shapes
group_rsm = np.load(BRAIN_RSM_PATH)
per_subj_rsms = np.load(PER_SUBJECT_RSMS_PATH)
print('group_rsm:', group_rsm.shape, '| per_subj_rsms:', per_subj_rsms.shape)


Brain RSMs exist.
group_rsm: (384, 384) | per_subj_rsms: (7, 384, 384)


## 4) Model feature extraction + RSA

In [7]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig
import torch, numpy as np, os, gc
from pathlib import Path
from scipy.stats import spearmanr

# ---------- pooling ----------
def _pool_layers(hidden_states, attention_mask):
    """
    hidden_states: tuple[L] of (B, T, H)
    attention_mask: (B, T)
    returns (B, L, H) mean-pooled per token with masking
    """
    mask = attention_mask.unsqueeze(-1).to(hidden_states[0].dtype)  # (B,T,1)
    denom = mask.sum(dim=1).clamp(min=1e-9)                         # (B,1)
    pooled = [ (h * mask).sum(dim=1) / denom for h in hidden_states ]  # list[L] of (B,H)
    return torch.stack(pooled, dim=1)                                # (B,L,H)

# ---------- model loader (low-RAM, GPU preferred) ----------
def get_model_and_tokenizer(model_name: str,
                            offload_dir: str = "/content/offload",
                            gpu_budget_gib: int = 13,
                            cpu_budget_gib: int = 4):
    """
    Loads model with safest possible memory settings:
      - 4-bit quantization if bitsandbytes GPU ops available, else fp16
      - device_map={'':0} to force GPU when present
      - low_cpu_mem_usage + offload_state_dict + max_memory + offload_folder
    """
    os.makedirs(offload_dir, exist_ok=True)

    # Tokenizer (robust padding for decoder models)
    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token if tok.eos_token is not None else tok.unk_token

    # Prefer AutoModel when possible, fall back to AutoModelForCausalLM for decoder-only LMs
    ModelCls = AutoModel
    try:
        # Some LLM repos expose only CausalLM heads; switch class if AutoModel fails later
        _ = AutoModel.from_pretrained  # marker
    except Exception:
        ModelCls = AutoModelForCausalLM

    # Bitsandbytes 4-bit if available
    quant_config = None
    bnb_ok = False
    try:
        import bitsandbytes as bnb
        _ = bnb.nn.Linear4bit
        bnb_ok = True
    except Exception:
        bnb_ok = False

    if torch.cuda.is_available():
        device_map = {"": 0}
    else:
        device_map = "cpu"

    max_memory = {
        0: f"{gpu_budget_gib}GiB" if torch.cuda.is_available() else "0GiB",
        "cpu": f"{cpu_budget_gib}GiB",
    }

    load_kwargs = dict(
        device_map=device_map,
        low_cpu_mem_usage=True,
        offload_state_dict=True,
        offload_folder=offload_dir,
        max_memory=max_memory,
        trust_remote_code=True,
        use_safetensors=True,
        output_hidden_states=True,
    )

    if bnb_ok and torch.cuda.is_available():
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )
        load_kwargs["quantization_config"] = quant_config
    else:
        if torch.cuda.is_available():
            load_kwargs["torch_dtype"] = torch.float16

    # Try base model first; if repo only supports CausalLM, fall back
    try:
        mdl = ModelCls.from_pretrained(model_name, **load_kwargs)
    except Exception:
        mdl = AutoModelForCausalLM.from_pretrained(model_name, **load_kwargs)

    return tok, mdl

# ---------- embeddings with batching and low memory ----------
def layerwise_sentence_embeddings(sentences, tokenizer, model, device,
                                  batch_size: int = 8, max_length: int = 256):
    """
    Returns np.ndarray of shape (N, L, H) in float32.
    Processes sentences in batches; pools token states per layer.
    """
    model.eval()
    N = len(sentences)
    all_layers = None
    hidden_size = None

    # First pass on a tiny batch to discover L and H without storing large tensors
    with torch.no_grad():
        enc0 = tokenizer(sentences[:1], return_tensors='pt', truncation=True,
                         padding=True, max_length=max_length)
        enc0 = {k: v.to(device) for k, v in enc0.items()}
        out0 = model(**enc0)
        L = len(out0.hidden_states)
        H = out0.hidden_states[0].shape[-1]
        del out0, enc0
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    reps = np.empty((N, L, H), dtype=np.float32)

    with torch.no_grad():
        for start in range(0, N, batch_size):
            end = min(N, start + batch_size)
            enc = tokenizer(sentences[start:end],
                            return_tensors='pt',
                            truncation=True,
                            padding=True,
                            max_length=max_length)
            enc = {k: v.to(device) for k, v in enc.items()}
            outp = model(**enc)  # returns hidden_states
            pooled = _pool_layers(outp.hidden_states, enc["attention_mask"])  # (B,L,H)
            reps[start:end] = pooled.detach().cpu().float().numpy()
            # free ASAP
            del outp, enc, pooled
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    return reps

# ---------- RSA pipeline (streamed, memory safe) ----------
def rsa_for_model(model_name,
                  sentences_path: Path = SENTENCES_PATH,
                  save_prefix: str | None = None,
                  batch_size: int = 8,
                  max_length: int = 256):
    """
    Loads model safely, computes layerwise sentence embeddings in batches,
    builds per-layer cosine RSMs, computes Spearman correlation to group brain RSM,
    and saves results under RESULTS_DIR. Returns (scores, rsm_stack).
    """
    # Read sentences
    sentences = [ln.strip() for ln in open(sentences_path, 'r', encoding='utf-8') if ln.strip()]
    assert len(sentences) == 384, f'Expected 384 sentences, got {len(sentences)}'

    # Model + tokenizer with safe memory settings
    tok, mdl = get_model_and_tokenizer(model_name)

    # Device comes from device_map; choose cuda:0 if present, else cpu
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'Loaded {model_name} on {device}.')

    # Compute embeddings efficiently
    reps = layerwise_sentence_embeddings(
        sentences, tok, mdl, device, batch_size=batch_size, max_length=max_length
    )  # shape (384, L, H)

    # Free model memory before RSA if needed
    del mdl, tok
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    N, L, H = reps.shape
    tri = np.triu_indices(N, 1)
    scores = np.empty(L, dtype=np.float32)
    rsm_stack = np.empty((L, N, N), dtype=np.float32)

    # Build per-layer RSMs and correlations
    for l in range(L):
        X = reps[:, l, :]                         # (N, H)
        rsm = compute_cosine_rsm(X)               # (N, N)
        rsm_stack[l] = rsm
        scores[l] = spearmanr(group_rsm[tri], rsm[tri]).statistic

    tag = save_prefix or model_name.split('/')[-1].replace('-', '_')
    np.save(RESULTS_DIR / f'{tag}_rsa_scores_expt2.npy', scores)
    np.save(RESULTS_DIR / f'{tag}_rsm_stack.npy', rsm_stack)
    print('Saved:', RESULTS_DIR / f'{tag}_rsa_scores_expt2.npy', 'and', RESULTS_DIR / f'{tag}_rsm_stack.npy')

    return scores, rsm_stack


## 5) Run models (uncomment as needed)

In [8]:
# # ==== GPU + bitsandbytes diagnostic + SAFE low-RAM loader for DeepSeek ====

# import os, sys, gc, subprocess, time
# from pathlib import Path

# print("== Torch / CUDA ==")
# import torch
# print("torch version         :", torch.__version__)
# print("cuda available        :", torch.cuda.is_available())
# print("cuda device count     :", torch.cuda.device_count())
# if torch.cuda.is_available():
#     print("cuda device name      :", torch.cuda.get_device_name(0))
#     try:
#         print("torch.version.cuda    :", torch.version.cuda)
#     except Exception:
#         pass

# print("\n== nvidia-smi ==")
# try:
#     out = subprocess.check_output(["nvidia-smi", "-L"]).decode()
#     print(out.strip() or "<no output>")
# except Exception as e:
#     print(f"nvidia-smi not available: {e}")

# # 2) Use Google Drive cache to avoid duplicates on /content
# BASE_DIR = "/content/drive/MyDrive/pereira2018"
# HF_HOME = os.path.join(BASE_DIR, "models", "huggingface")
# os.makedirs(HF_HOME, exist_ok=True)
# os.environ["HF_HOME"] = HF_HOME
# os.environ["TRANSFORMERS_CACHE"] = HF_HOME
# os.environ["HF_DATASETS_CACHE"] = os.path.join(HF_HOME, "datasets")
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

# # 3) bitsandbytes check (install if missing)
# print("\n== bitsandbytes ==")
# bnb_ok = False
# try:
#     import bitsandbytes as bnb
#     _ = bnb.nn.Linear4bit
#     bnb_ok = True
#     print("bitsandbytes GPU ops  : available")
# except Exception as e:
#     print("bitsandbytes issue    :", e)
#     print("Trying to install/repair bitsandbytes for CUDA...")
#     !pip -q install --upgrade bitsandbytes accelerate
#     import importlib
#     try:
#         bnb = importlib.import_module("bitsandbytes")
#         _ = bnb.nn.Linear4bit
#         bnb_ok = True
#         print("bitsandbytes GPU ops  : available after install")
#     except Exception as e2:
#         print("bitsandbytes still not ready for GPU:", e2)

# # 4) Load model SAFELY: cap RAM, stream state dict, offload overflow to disk
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# MODEL_ID = "deepseek-ai/deepseek-llm-7b-base"  # change if needed
# OFFLOAD_DIR = "/content/offload"
# os.makedirs(OFFLOAD_DIR, exist_ok=True)

# # Budgets to prevent RAM crash (adjust if needed)
# gpu_budget_gib = 13     # on T4 ~15 GiB total; keep headroom
# cpu_budget_gib = 4      # keep well under free-tier RAM

# max_memory = {
#     0: f"{gpu_budget_gib}GiB",
#     "cpu": f"{cpu_budget_gib}GiB",
# }
# print("\n== Loader budgets ==")
# print("max_memory:", max_memory, "| offload_folder:", OFFLOAD_DIR)

# quant_config = None
# if bnb_ok and torch.cuda.is_available():
#     quant_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_compute_dtype=torch.float16,
#     )

# print("\n== Loading tokenizer/model ==")
# tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# load_kwargs = dict(
#     device_map={"": 0} if torch.cuda.is_available() else "cpu",  # force GPU placement
#     max_memory=max_memory,               # respect CPU/GPU memory caps
#     offload_folder=OFFLOAD_DIR,          # spill extra weights to disk, not RAM
#     offload_state_dict=True,             # stream shards during load
#     low_cpu_mem_usage=True,              # stream tensors into modules
#     use_safetensors=True,                # prefer mmap'd format
#     trust_remote_code=True,
# )

# if quant_config is not None:
#     load_kwargs["quantization_config"] = quant_config
# else:
#     if torch.cuda.is_available():
#         load_kwargs["torch_dtype"] = torch.float16

# model = AutoModelForCausalLM.from_pretrained(MODEL_ID, **load_kwargs)

# # 5) Inspect actual placement
# device_map = getattr(model, "hf_device_map", None)
# print("\n== hf_device_map ==")
# print(device_map if device_map is not None else "<no device map attribute>")

# def gpu_mem():
#     if not torch.cuda.is_available():
#         return "no cuda"
#     torch.cuda.synchronize()
#     return f"alloc={torch.cuda.memory_allocated(0)/1e9:.2f} GB, reserved={torch.cuda.memory_reserved(0)/1e9:.2f} GB"

# print("GPU memory after load:", gpu_mem())

# # 6) Sanity forward pass (keeps model resident so you can observe VRAM)
# text = "Hello from DeepSeek on GPU."
# inputs = tokenizer(text, return_tensors="pt")
# if torch.cuda.is_available():
#     inputs = {k: v.to(0) for k, v in inputs.items()}

# with torch.inference_mode():
#     _ = model.generate(**inputs, max_new_tokens=64)

# print("GPU memory during use:", gpu_mem())
# print("Sleeping 20s; run `!nvidia-smi` in another cell to inspect VRAM...")
# time.sleep(20)

# # 7) Cleanup (uncomment when done)
# # del model, tokenizer, inputs, _
# # gc.collect()
# # if torch.cuda.is_available():
# #     torch.cuda.empty_cache()
# # print("GPU memory after cleanup:", gpu_mem())


== Torch / CUDA ==
torch version         : 2.8.0+cu126
cuda available        : False
cuda device count     : 0

== nvidia-smi ==
nvidia-smi not available: [Errno 2] No such file or directory: 'nvidia-smi'

== bitsandbytes ==
bitsandbytes issue    : No module named 'bitsandbytes'
Trying to install/repair bitsandbytes for CUDA...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.5 MB/s eta 0:00:00
bitsandbytes GPU ops  : available after install

== Loader budgets ==
max_memory: {0: '13GiB', 'cpu': '4GiB'} | offload_folder: /content/offload

== Loading tokenizer/model ==


tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# --- Run ONE model at a time. Toggle as needed. ---

import os, gc, torch
from huggingface_hub import login

# Hugging Face login (optional: required only for gated models)
HF_TOKEN = os.environ.get("HF_TOKEN", "").strip()
if HF_TOKEN:
    try:
        login(token=HF_TOKEN, add_to_git_credential=False)
        print("✅ Hugging Face login OK.")
    except Exception as e:
        print(f"[WARN] HF login failed: {e}")
else:
    print("ℹ️ No HF_TOKEN set; public models will work, gated ones may 403.")


def _cleanup(*objs):
    for o in objs:
        try:
            del o
        except Exception:
            pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


# ================================
# ✅ MISTRAL-7B (OPEN, RUN THIS)
# ================================
# try:
#     mistral_model_id = "mistralai/Mistral-7B-v0.1"  # or "mistralai/Mistral-7B-Instruct-v0.3"
#     mistral_scores, mistral_rsms = rsa_for_model(
#         mistral_model_id,
#         save_prefix='mistral_7b'
#     )
#     print("Saved: results/mistral_7b_rsa_scores_expt2.npy  &  results/mistral_7b_rsm_stack.npy")
# except Exception as e:
#     print("❌ Mistral run failed:", e)
# finally:
#     _cleanup(mistral_scores if 'mistral_scores' in locals() else None,
#              mistral_rsms   if 'mistral_rsms'   in locals() else None)


# ==================================================
# ✅ GPT-2 (OPTIONAL: small, fast sanity check)
# ==================================================
# try:
#     gpt2_scores, gpt2_rsms = rsa_for_model('gpt2', save_prefix='gpt2')
#     print("Saved: results/gpt2_rsa_scores_expt2.npy  &  results/gpt2_rsm_stack.npy")
# finally:
#     _cleanup()


# ==================================================
# ✅ DEEPSEEK-7B (OPTIONAL: heavy, memory-critical)
# ==================================================
# try:
#     deepseek_model_id = 'deepseek-ai/deepseek-llm-7b-base'
#     deepseek_scores, deepseek_rsms = rsa_for_model(
#         deepseek_model_id,
#         save_prefix='deepseek_7b'
#     )
#     print("Saved: results/deepseek_7b_rsa_scores_expt2.npy  &  results/deepseek_7b_rsm_stack.npy")
# except Exception as e:
#     print("❌ DeepSeek run failed:", e)
# finally:
#     _cleanup(deepseek_scores if 'deepseek_scores' in locals() else None,
#              deepseek_rsms   if 'deepseek_rsms'   in locals() else None)


# ==================================================
# ✅ LLaMA-2-7B (OPTIONAL: requires Meta approval)
# ==================================================
try:
    llama_model_id = 'meta-llama/Llama-2-7b-hf'
    llama_scores, llama_rsms = rsa_for_model(
        llama_model_id,
        save_prefix='llama2_7b'
    )
    print("Saved: results/llama2_7b_rsa_scores_expt2.npy  &  results/llama2_7b_rsm_stack.npy")
except Exception as e:
    print("❌ LLaMA-2 run failed:", e)
finally:
    _cleanup(llama_scores if 'llama_scores' in locals() else None,
             llama_rsms   if 'llama_rsms'   in locals() else None)


ℹ️ No HF_TOKEN set; public models will work, gated ones may 403.


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
###

# ============================
# Visualization of saved RSA results
# ============================

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import re

# ---- Paths ----
BASE_DIR = Path('/content/drive/MyDrive/pereira2018')
RESULTS_DIR = BASE_DIR / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ---- Utilities ----
def _model_tag_from(filename: str) -> str:
    # Extract prefix before known suffixes
    # e.g., mistral_7b_rsa_scores_expt2.npy -> mistral_7b
    return re.sub(r'_(rsa_scores_expt2|rsm_stack|layerwise_.*)\.npy$', '', filename)

def _discover_models(results_dir: Path):
    scores = {}
    rsms = {}
    stats = {}  # optional: obs_means, p_means, fdr_sig
    for p in results_dir.glob('*_rsa_scores_expt2.npy'):
        tag = _model_tag_from(p.name)
        scores[tag] = p
    for p in results_dir.glob('*_rsm_stack.npy'):
        tag = _model_tag_from(p.name)
        rsms[tag] = p
    # Optional stats saved earlier
    for tag in scores.keys():
        obs = results_dir / f'{tag}_layerwise_obs_means.npy'
        pv  = results_dir / f'{tag}_layerwise_p_means.npy'
        sig = results_dir / f'{tag}_layerwise_fdr_sig.npy'
        if obs.exists() and pv.exists() and sig.exists():
            stats[tag] = {'obs': obs, 'p': pv, 'sig': sig}
    return scores, rsms, stats

def _load_optional(path: Path):
    return np.load(path) if path and path.exists() else None

def _plot_layerwise_curves(scores_dict, stats_dict):
    plt.figure(figsize=(9, 5))
    peak_info = {}
    for tag, p in scores_dict.items():
        s = np.load(p)  # shape (L,)
        L = s.shape[0]
        x = np.arange(L)
        plt.plot(x, s, label=tag)
        # Peak info
        li = int(np.argmax(s))
        peak_info[tag] = {'layer': li, 'score': float(s[li])}
        # Overlay significance if available
        st = stats_dict.get(tag)
        if st:
            sig = np.load(st['sig']).astype(bool)
            sig_idx = np.where(sig)[0]
            if sig_idx.size > 0:
                plt.scatter(sig_idx, s[sig_idx], marker='o', s=20)
    plt.xlabel('Layer')
    plt.ylabel('RSA to brain (Spearman)')
    plt.title('Layerwise brain alignment across models')
    plt.legend()
    out = RESULTS_DIR / 'viz_layerwise_rsa.png'
    plt.tight_layout()
    plt.savefig(out, dpi=150)
    plt.show()
    print('Saved:', out)
    return peak_info

def _plot_peak_rsms(rsms_dict, peak_info, n_cols=3):
    # Collect RSM heatmaps at the peak layer for each model
    items = []
    for tag, p in rsms_dict.items():
        if tag not in peak_info:
            continue
        stack = np.load(p)  # (L, 384, 384)
        li = int(peak_info[tag]['layer'])
        items.append((tag, li, stack[li]))
    if not items:
        print('No RSM stacks found to plot.')
        return
    n = len(items)
    n_cols = min(n_cols, n)
    n_rows = (n + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 4*n_rows))
    if n_rows == 1 and n_cols == 1:
        axes = np.array([[axes]])
    elif n_rows == 1:
        axes = np.array([axes])

    for idx, (tag, li, rsm) in enumerate(items):
        r = idx // n_cols
        c = idx % n_cols
        ax = axes[r, c]
        im = ax.imshow(rsm, interpolation='nearest')
        ax.set_title(f'{tag} • peak layer {li}')
        ax.set_xticks([]); ax.set_yticks([])
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    # Hide unused axes
    for k in range(len(items), n_rows*n_cols):
        r = k // n_cols
        c = k % n_cols
        axes[r, c].axis('off')

    out = RESULTS_DIR / 'viz_peak_rsms.png'
    plt.tight_layout()
    plt.savefig(out, dpi=150)
    plt.show()
    print('Saved:', out)

def _plot_peak_bars(peak_info):
    if not peak_info:
        print('No peak info available.')
        return
    tags = list(peak_info.keys())
    vals = [peak_info[t]['score'] for t in tags]
    plt.figure(figsize=(7, 4))
    plt.bar(tags, vals)
    plt.ylabel('Peak RSA (Spearman)')
    plt.title('Peak brain alignment by model')
    plt.xticks(rotation=20, ha='right')
    out = RESULTS_DIR / 'viz_peak_bars.png'
    plt.tight_layout()
    plt.savefig(out, dpi=150)
    plt.show()
    print('Saved:', out)

# ---- Run visualization ----
scores_dict, rsms_dict, stats_dict = _discover_models(RESULTS_DIR)
if not scores_dict:
    raise RuntimeError(f'No *_rsa_scores_expt2.npy files found in {RESULTS_DIR}')

print('Discovered models:', ', '.join(sorted(scores_dict.keys())))
peak_info = _plot_layerwise_curves(scores_dict, stats_dict)
_plot_peak_bars(peak_info)
_plot_peak_rsms(rsms_dict, peak_info)




## 6) Subject-level stats & FDR

In [ ]:
# =========================
# Subject-level stats & FDR
# =========================
# Consistent 4-space indentation, no tabs.

import numpy as np
from scipy.stats import spearmanr

def _upper_tri(x: np.ndarray):
    """Return upper-triangular (i<j) vector of a square matrix."""
    n = x.shape[0]
    tri = np.triu_indices(n, 1)
    return x[tri]

def permutation_pvals(model_rsm: np.ndarray,
                      per_subj_rsms: np.ndarray,
                      n_perm: int = 2000,
                      seed: int = 0):
    """
    Compute observed mean Spearman rho across subjects between model_rsm and per-subject RSMs,
    then a permutation distribution by shuffling sentence order (rows & cols together).
    Returns:
        obs_mean: float
        pval: float (one-sided; probability perm >= obs)
    """
    rng = np.random.default_rng(seed)
    # Observed mean rho across subjects
    m_vec = _upper_tri(model_rsm)
    obs_rhos = []
    for s in range(per_subj_rsms.shape[0]):
        r_vec = _upper_tri(per_subj_rsms[s])
        obs_rhos.append(spearmanr(m_vec, r_vec).statistic)
    obs_mean = float(np.mean(obs_rhos))

    # Permutations
    n = model_rsm.shape[0]
    perm_means = np.empty(n_perm, dtype=np.float32)
    for i in range(n_perm):
        perm = rng.permutation(n)
        # Apply the same permutation to rows and columns
        m_perm = model_rsm[perm][:, perm]
        m_perm_vec = _upper_tri(m_perm)
        rhos = []
        for s in range(per_subj_rsms.shape[0]):
            r_vec = _upper_tri(per_subj_rsms[s])
            rhos.append(spearmanr(m_perm_vec, r_vec).statistic)
        perm_means[i] = np.mean(rhos)

    # One-sided p-value: fraction of permutations >= observed
    pval = float((perm_means >= obs_mean).mean())
    return obs_mean, pval

def fdr_bh(pvals: np.ndarray, alpha: float = 0.05):
    """
    Benjamini–Hochberg procedure.
    Returns a boolean array indicating significance at FDR alpha.
    """
    pvals = np.asarray(pvals, dtype=float)
    m = pvals.size
    order = np.argsort(pvals)
    ranked = pvals[order]
    thresholds = alpha * (np.arange(1, m + 1) / m)
    passed = ranked <= thresholds
    if not np.any(passed):
        return np.zeros_like(pvals, dtype=bool)
    k_max = np.max(np.where(passed)[0])  # last index that passed
    cutoff = ranked[k_max]
    return pvals <= cutoff

def layerwise_subject_stats(rsm_stack: np.ndarray,
                            per_subj_rsms: np.ndarray,
                            n_perm: int = 2000,
                            seed: int = 0):
    """
    For each layer’s model RSM, compute:
      - observed mean rho across subjects
      - permutation p-value
    Then apply BH-FDR across layers.
    Returns:
      obs_means: (L,) array
      p_means:   (L,) array of p-values
      sig:       (L,) boolean array (FDR at alpha=0.05)
    """
    L = rsm_stack.shape[0]
    obs_means, p_means = [], []
    # Different random seeds per layer to vary permutations reproducibly
    for l in range(L):
        obs, p = permutation_pvals(rsm_stack[l], per_subj_rsms, n_perm=n_perm, seed=seed + l)
        obs_means.append(obs)
        p_means.append(p)
    obs_means = np.asarray(obs_means, dtype=float)
    p_means = np.asarray(p_means, dtype=float)
    sig = fdr_bh(p_means, alpha=0.05)
    return obs_means, p_means, sig


gpt2_obs_mean, gpt2_p_mean, gpt2_sig = layerwise_subject_stats(
    gpt2_rsms,           # shape (L, 384, 384)
    per_subj_rsms,       # shape (S, 384, 384)
    n_perm=2000,
    seed=0
)


## 7) Noise ceiling (split-half)

In [ ]:

def split_half_noise_ceiling(per_subj_rsms, B=1000, seed=0):
    S = per_subj_rsms.shape[0]; tri = np.triu_indices(384,1)
    rng = np.random.default_rng(seed); vals = []
    for _ in range(B):
        idx = rng.permutation(S)
        A = per_subj_rsms[idx[:S//2]].mean(axis=0)
        Bm = per_subj_rsms[idx[S//2:]].mean(axis=0)
        vals.append(spearmanr(A[tri], Bm[tri]).statistic)
    return float(np.mean(vals)), tuple(np.percentile(vals, [2.5,97.5]))


## 8) Plot curves

In [ ]:

import matplotlib.pyplot as plt, numpy as np
def load_if_exists(p):
    return np.load(p) if Path(p).exists() else None

gpt2 = load_if_exists(RESULTS_DIR/'gpt2_rsa_scores_expt2.npy')
deepseek = load_if_exists(RESULTS_DIR/'deepseek_7b_rsa_scores_expt2.npy')
llama = load_if_exists(RESULTS_DIR/'llama2_7b_rsa_scores_expt2.npy')

plt.figure()
if gpt2 is not None: plt.plot(gpt2, label='GPT-2')
if deepseek is not None: plt.plot(deepseek, label='DeepSeek-7B')
if llama is not None: plt.plot(llama, label='LLaMA-2-7B')
plt.xlabel('Layer'); plt.ylabel('Spearman ρ with Brain RSM')
plt.title('Layer-wise RSA Comparison')
plt.legend(); plt.show()


## 10) Stimuli alignment & hashing (sanity check)

In [ ]:
import hashlib, json

def read_sentences(path):
    sents = [ln.strip() for ln in open(path, 'r', encoding='utf-8') if ln.strip()]
    assert len(sents)==384, f'Expected 384 sentences, got {len(sents)}'
    return sents

def hash_sentences(sents):
    h = hashlib.sha256(("\n".join(sents)).encode('utf-8')).hexdigest()
    return h

_sents = read_sentences(SENTENCES_PATH)
stimuli_hash = hash_sentences(_sents)
print("Stimuli hash (SHA-256):", stimuli_hash)


## 11) Embedding cache (skip recompute)

In [ ]:

import numpy as np

def cache_path_for(model_tag: str):
    return RESULTS_DIR / f'{model_tag}_layerwise_sentence_embeddings.npy'

def save_embeddings(model_tag: str, arr: np.ndarray):
    np.save(cache_path_for(model_tag), arr)
    print("Cached embeddings:", cache_path_for(model_tag))

def load_embeddings_if_any(model_tag: str):
    p = cache_path_for(model_tag)
    return np.load(p) if p.exists() else None


## 12) Token pooling ablations (mean / last / attention-weighted)

In [ ]:

import torch

def pooled_hidden_states(hidden_states, attention_mask=None, mode='mean'):
    pooled = []
    for h in hidden_states:  # (1, T, D)
        if mode == 'mean':
            v = h.mean(dim=1)
        elif mode == 'last':
            v = h[:, -1, :]
        elif mode == 'attn':
            if attention_mask is None:
                v = h.mean(dim=1)
            else:
                m = attention_mask.unsqueeze(-1).float()
                den = m.sum(dim=1).clamp(min=1e-9)
                v = (h * m).sum(dim=1) / den
        else:
            raise ValueError("Unknown mode")
        pooled.append(v.squeeze(0))
    return torch.stack(pooled, dim=0)

def layerwise_sentence_embeddings_with_mode(sentences, tokenizer, model, device, mode='mean'):
    model.eval()
    model = to_device_half_if_possible(model, device)
    out = []
    with torch.no_grad():
        for s in sentences:
            enc = tokenizer(s, return_tensors='pt', truncation=True)
            enc = {k: v.to(device) for k, v in enc.items()}
            outp = model(**enc)
            pooled = pooled_hidden_states(outp.hidden_states, enc.get('attention_mask', None), mode=mode)
            out.append(pooled.cpu().float().numpy())
    return np.stack(out, axis=0)


## 13) Linear CKA as a robustness metric

In [ ]:

import numpy as np

def center_gram(K):
    n = K.shape[0]
    H = np.eye(n) - np.ones((n, n))/n
    return H @ K @ H

def linear_kernel(X):
    return X @ X.T

def linear_cka(X, Y):
    Kx = center_gram(linear_kernel(X))
    Ky = center_gram(linear_kernel(Y))
    hsic = (Kx * Ky).sum()
    norm = np.sqrt((Kx*Kx).sum() * (Ky*Ky).sum())
    return float(hsic / (norm + 1e-12))


## 14) Bootstrap confidence intervals for RSA curves (subject-resampling)

In [ ]:

from scipy.stats import spearmanr

def rsa_curve_bootstrap_ci(rsm_stack, per_subj_rsms, B=1000, seed=0):
    # rsm_stack: (L, 384, 384)
    rng = np.random.default_rng(seed)
    tri = np.triu_indices(384,1)
    S = per_subj_rsms.shape[0]
    L = rsm_stack.shape[0]
    boot = np.zeros((B, L), dtype=float)
    for b in range(B):
        idx = rng.integers(0, S, size=S)  # bootstrap subjects
        grp = per_subj_rsms[idx].mean(axis=0)
        for l in range(L):
            boot[b, l] = spearmanr(grp[tri], rsm_stack[l][tri]).statistic
    mean = boot.mean(axis=0)
    lo, hi = np.percentile(boot, [2.5, 97.5], axis=0)
    return mean, lo, hi


## 15) Encoding models (ridge CV) — whole-brain or ROI if masks provided

In [ ]:

from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

# Helpers to get subject-level voxel matrices X_brain (384 × V)
def load_subject_beta_matrix(subj_dir: Path):
    mat = next((p for p in [subj_dir/'examples_384sentences.mat', subj_dir/'data_384sentences.mat'] if p.exists()), None)
    if not mat:
        return None
    X = extract_384_matrix(mat)  # (384, V)
    return X

def layer_features_from_cache_or_model(model_tag, layer_idx):
    E = load_embeddings_if_any(model_tag)
    if E is None:
        raise FileNotFoundError(f'No cached embeddings for {model_tag}. Run rsa_for_model() first; it will save embeddings if you modify it to call save_embeddings()')
    return E[:, layer_idx, :]  # (384, D)

def ridge_encoding_cv(X_feat, Y_vox, alphas=(1e0, 1e1, 1e2, 1e3), n_splits=5, seed=0):
    # Standardize both X and Y within train folds
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    r2s = []
    for tr, te in kf.split(X_feat):
        Xtr, Xte = X_feat[tr], X_feat[te]
        Ytr, Yte = Y_vox[tr], Y_vox[te]
        Xs = StandardScaler(with_mean=True, with_std=True).fit_transform(Xtr)
        Xt = StandardScaler(with_mean=True, with_std=True).fit(Xtr).transform(Xte)  # match train scaling
        # Fit ridge per voxel via multi-target regression
        model = RidgeCV(alphas=alphas, fit_intercept=True, scoring=None)  # we'll compute R^2 manually
        model.fit(Xs, Ytr)
        Yhat = model.predict(Xt)
        # CV R^2 per voxel, then mean across voxels
        ss_res = ((Yte - Yhat)**2).sum(axis=0)
        ss_tot = ((Yte - Yte.mean(axis=0))**2).sum(axis=0) + 1e-12
        r2 = 1 - ss_res/ss_tot
        r2s.append(np.mean(r2))
    return float(np.mean(r2s))

# Example workflow (after caching embeddings and having RAW_DIR):
# subj_dirs = [d for d in RAW_DIR.glob('*') if d.is_dir()]
# model_tag = 'gpt2'
# layer = 8
# X_feat = layer_features_from_cache_or_model(model_tag, layer)  # (384, D)
# r2_scores = []
# for sd in subj_dirs:
#     Y_vox = load_subject_beta_matrix(sd)  # (384, V)
#     r2_scores.append(ridge_encoding_cv(X_feat, Y_vox))
# print('Mean CV-R^2 across subjects:', np.mean(r2_scores))


## 16) Reproducibility — environment capture

In [ ]:

import sys, subprocess, json, platform, time

def capture_environment(out_json=RESULTS_DIR/'env_capture.json'):
    info = {
        "python_version": sys.version,
        "platform": platform.platform(),
        "time_utc": time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())
    }
    try:
        pkgs = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
        info["pip_freeze"] = pkgs.splitlines()
    except Exception as e:
        info["pip_freeze_error"] = str(e)
    out_json.write_text(json.dumps(info, indent=2))
    print("Saved environment:", out_json)

# capture_environment()


## 17) Plot with error bands (if bootstraps available)

In [ ]:

import matplotlib.pyplot as plt

def plot_curves_with_bands(scores_dict, ci_dict=None, title='Layer-wise RSA'):
    plt.figure()
    for label, y in scores_dict.items():
        plt.plot(y, label=label)
        if ci_dict and label in ci_dict:
            lo, hi = ci_dict[label]
            xs = np.arange(len(y))
            plt.fill_between(xs, lo, hi, alpha=0.2)
    plt.xlabel('Layer')
    plt.ylabel('Spearman ρ with Brain RSM')
    plt.title(title)
    plt.legend()
    plt.show()
